In [ ]:
import sys
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
sys.path.append(os.path.join(os.pardir, 'gotmtool'))
from gotmtool import *
from gotmtool.map import *

In [ ]:
gotmtool_config = os.path.join(os.pardir, 'gotmtool', '.gotm_env.yaml')
gotm_env = config_load(gotmtool_config)

In [ ]:
casename = 'JRA55-do_Global_dampV5d_3h'
dz = 1
dt = 600
simtime = {
    '20090101': '20090131',
    '20090201': '20090228',
    '20090301': '20090331',
    '20090401': '20090430',
    '20090501': '20090531',
    '20080601': '20080630',
    '20080701': '20080731',
    '20080801': '20080831',
    '20080901': '20080930',
    '20081001': '20081031',
    '20081101': '20081130',
    '20081201': '20081231',
}
month_labels = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',\
                'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
turbmethods = ['KPP-CVMix', 'KPPLT-LF17', 'StokesMOST']
figroot = os.path.join(gotm_env['gotmdir_figure'], casename)
os.makedirs(figroot, exist_ok=True)

In [ ]:
gmobj = {}
gmobj_mask = {}
for date_start in simtime.keys():
# for date_start in ['20080601']:
    date_end = simtime[date_start]
    setup = 'VR{:d}m_DT{:d}s_{:s}-{:s}'.format(int(dz), int(dt), date_start, date_end)
    print(setup)
    inputdir = os.path.join(gotm_env['gotmdir_figure'], 'data', casename, setup)
    tmp = {}
    for turbmethod in turbmethods:
        inputfile = 'data_map_mld_deltaR_mean_{:s}.npz'.format(turbmethod)
        tmp[turbmethod] = GOTMMap().load(path=os.path.join(inputdir, inputfile))
    gmobj[date_start] = tmp
    dref = tmp['KPP-CVMix']
    dmask = np.where(dref.data>450, 0, 1)
    gmobj_mask[date_start] = GOTMMap(data=dmask, lon=dref.lon, lat=dref.lat, name='mask', units='1:valid,0:invalid')


In [ ]:
turbmethod = 'KPP-CVMix'

# plot figure
f, axarr = plt.subplots(6, 2, sharex='col', subplot_kw={'projection': ccrs.PlateCarree(central_longitude=200)})
f.set_size_inches(12, 13.5)

# plot
levels0 = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 110, 130, \
           150, 180, 210, 240, 280, 320, 360, 407, 454, 500]
for j, date_start in enumerate(simtime.keys()):
    m = j//2
    n = j%2
    ax = axarr[m,n]
    mlabel = month_labels[j]
    gm = gmobj[date_start][turbmethod].masked(gmobj_mask[date_start])
    im0 = gm.plot(ax=ax, levels=levels0, add_colorbar=False, cmap='viridis')
    # label
    ax.text(0.1, 0.9, mlabel, fontsize=14, color='k', fontweight='bold', va='top', ha='left', transform=ax.transAxes,
                    bbox=dict(boxstyle="square",ec='k',fc='w'))

# reduce margin
plt.tight_layout()

# colorbar
cb_ticks = [0, 30, 60, 90, 150, 240, 360, 500]
plt.subplots_adjust(right=0.95)
cax0 = plt.axes([0.85, 0.15, 0.1, 0.7])
cax0.set_visible(False)
cb0 = plt.colorbar(im0, ax=cax0, ticks=cb_ticks, aspect=35)

# save figure
figname = os.path.join(figroot,'fig_mld_deltaR_{:s}.png'.format(turbmethod))
plt.savefig(figname, dpi = 300)

In [ ]:
turbmethod_ref = 'KPP-CVMix'
turbmethod = 'StokesMOST'

# plot figure
f, axarr = plt.subplots(6, 2, sharex='col', subplot_kw={'projection': ccrs.PlateCarree(central_longitude=200)})
f.set_size_inches(12, 13.5)

# plot
levels1 = [-70, -60, -50, -40, -30, -20, -18, -16, -14, -12, \
           -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, \
           12, 14, 16, 18, 20, 30, 40, 50, 60, 70] 
for j, date_start in enumerate(simtime.keys()):
    m = j//2
    n = j%2
    ax = axarr[m,n]
    mlabel = month_labels[j]
    gm = (gmobj[date_start][turbmethod]-gmobj[date_start][turbmethod_ref].masked(gmobj_mask[date_start]))
    im0 = gm.plot(ax=ax, levels=levels1, add_colorbar=False, cmap='RdBu_r')
    # label
    ax.text(0.1, 0.9, mlabel, fontsize=14, color='k', fontweight='bold', va='top', ha='left', transform=ax.transAxes,
                    bbox=dict(boxstyle="square",ec='k',fc='w'))

# reduce margin
plt.tight_layout()

# colorbar
cb_ticks = [-60, -40, -20, -16, -12, -8, -4, 0, 4, 8, 12, 16, 20, 40, 60]
plt.subplots_adjust(right=0.95)
cax0 = plt.axes([0.85, 0.15, 0.1, 0.7])
cax0.set_visible(False)
cb0 = plt.colorbar(im0, ax=cax0, ticks=cb_ticks, aspect=35)

# save figure
figname = os.path.join(figroot,'fig_mld_deltaR_diff_{:s}_{:s}.png'.format(turbmethod, turbmethod_ref))
plt.savefig(figname, dpi = 300)

In [ ]:
turbmethod_ref = 'KPP-CVMix'
turbmethod = 'StokesMOST'

# plot figure
f, axarr = plt.subplots(6, 2, sharex='col', subplot_kw={'projection': ccrs.PlateCarree(central_longitude=200)})
f.set_size_inches(12, 13.5)

# plot
levels = list(np.arange(41))
for j, date_start in enumerate(simtime.keys()):
    m = j//2
    n = j%2
    ax = axarr[m,n]
    mlabel = month_labels[j]
    gm = (gmobj[date_start][turbmethod]-gmobj[date_start][turbmethod_ref].masked(gmobj_mask[date_start]))/gmobj[date_start][turbmethod_ref]*100.
    im0 = gm.plot(ax=ax, levels=levels, add_colorbar=False, cmap='afmhot_r')
    # label
    ax.text(0.1, 0.9, mlabel, fontsize=14, color='k', fontweight='bold', va='top', ha='left', transform=ax.transAxes,
                    bbox=dict(boxstyle="square",ec='k',fc='w'))

# reduce margin
plt.tight_layout()

# colorbar
cb_ticks = list(np.arange(5)*10)
cb_ticks_labels = ['{:d} %'.format(cb_ticks[i]) for i in np.arange(5)]
plt.subplots_adjust(right=0.95)
cax0 = plt.axes([0.85, 0.15, 0.1, 0.7])
cax0.set_visible(False)
cb0 = plt.colorbar(im0, ax=cax0, ticks=cb_ticks, aspect=35)
cb0.ax.set_yticklabels(cb_ticks_labels)

# save figure
figname = os.path.join(figroot,'fig_mld_deltaR_pchg_{:s}_{:s}.png'.format(turbmethod, turbmethod_ref))
plt.savefig(figname, dpi = 300)